In [50]:
import pandas as pd
import re
import string
import numpy as np
import nltk
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import spacy
from sklearn.naive_bayes import MultinomialNB
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
from sklearn.model_selection import train_test_split

from spacy_download import load_spacy
# Will download the model if it isn't installed yet
#nlp = load_spacy("en_core_web_sm", exclude=["parser", "tagger"]) 
nlp = spacy.load("en_core_web_sm")

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn import svm
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import KFold

In [51]:
df_train = pd.read_csv('C:/Users/Dikshant Gupta/Desktop/VIT/SEM 6/CSE3020_DV_Prj_NIL/data/train_data_cleaning2.csv')
df_test = pd.read_csv('C:/Users/Dikshant Gupta/Desktop/VIT/SEM 6/CSE3020_DV_Prj_NIL/data/test_data_cleaning2.csv')
df_train.head(5)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this # earthquake...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask . Canada,1
2,5,NaN,NaN,All residents asked to ' shelter in place ' ...,1
3,6,NaN,NaN,"13,000 people receive # wildfires evacuation ...",1
4,7,NaN,NaN,Just got sent this photo from Ruby # Alaska a...,1


In [52]:
df_train.isnull().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [53]:
df_train.shape

(7613, 5)

# Pre-Processing

In [54]:
def text_cleaner(text):
    text = text.lower() # convert to lowercase
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text) # remove punctuation and special characters
    text = re.sub(r'\s+', ' ', text) # remove extra whitespace
    text = re.sub(r'https?://\S+', '', text) # remove URLs
    text = re.sub(r"#", "", text)
    return text

In [55]:
df_train['new'] = df_train['text'].apply(text_cleaner)

In [56]:
df_train['new_text']=df_train['new'].apply(lambda x:nlp(x))
df_train.head()
df_train['new_text']

0       (our, deeds, are, the, reason, of, this, earth...
1           (forest, fire, near, la, ronge, sask, canada)
2       (all, residents, asked, to, shelter, in, place...
3       (13000, people, receive, wildfires, evacuation...
4       (just, got, sent, this, photo, from, ruby, ala...
                              ...                        
7608    (two, giant, cranes, holding, a, bridge, colla...
7609    ( , aria, ahrary, thetawniest, the, out, of, c...
7610    (m1, 94, 01, 04, utc, 5, km, s, of, volcano, h...
7611    (police, investigating, after, an, e, bike, co...
7612    (the, latest, more, homes, razed, by, northern...
Name: new_text, Length: 7613, dtype: object

In [57]:
def transform(text):
    doc=nlp(text,disable=['parser','ner'])
    lemmas=[token.lemma_ for token in doc]
    a_lemmas=[lemma for lemma in lemmas if lemma.isalpha()]
    return ' '.join(a_lemmas)

In [58]:
df_train['new_text']=df_train['new_text'].apply(transform)
df_train['new_text']

0       our deed be the reason of this earthquake may ...
1                   forest fire near la ronge sask canada
2       all resident ask to shelter in place be be not...
3       people receive wildfire evacuation order in ca...
4       just got send this photo from ruby alaska as s...
                              ...                        
7608    two giant crane hold a bridge collapse into ne...
7609    aria ahrary thetawniest the out of control wil...
7610                           utc km s of volcano hawaii
7611    police investigate after an e bike collide wit...
7612    the late more home raze by northern california...
Name: new_text, Length: 7613, dtype: object

In [59]:
X=df_train['new_text']
Y=df_train['target']
del df_train['new']
df_train.shape

(7613, 6)

In [60]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,random_state=4,test_size=0.2,stratify=Y)

In [61]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer=TfidfVectorizer(stop_words='english')
X_train=vectorizer.fit_transform(X_train)
X_test=vectorizer.transform(X_test)

# Model Building & Testing

In [62]:
models = []
models.append(('Logistic Regression', LogisticRegression()))
models.append(('Kernel SVM', SVC()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('Multinomial NB', MultinomialNB()))
models.append(('Random Forest', RandomForestClassifier()))
models.append(('Decision Tree Classifier', DecisionTreeClassifier()))
models

[('Logistic Regression', LogisticRegression()),
 ('Kernel SVM', SVC()),
 ('KNN', KNeighborsClassifier()),
 ('Multinomial NB', MultinomialNB()),
 ('Random Forest', RandomForestClassifier()),
 ('Decision Tree Classifier', DecisionTreeClassifier())]

In [63]:
X_train.shape

(6090, 11527)

In [64]:
from sklearn import model_selection
acc_score=[]
auc_score=[]
pre_results=[]
f1_results=[]
names=[]
result_col=pd.DataFrame(columns=['Algorithm','accuracy','ROC','Precision','f1_score'])
result_col

i=0
for name,model in models:
    kfold=model_selection.KFold(n_splits=10)
    names.append(name)
    cv_acc_results = model_selection.cross_val_score(model, X_train, Y_train,cv=kfold,scoring="accuracy")
    cv_auc_results = model_selection.cross_val_score(model, X_train, Y_train,cv=kfold,scoring="roc_auc")
    cv_pre_results = model_selection.cross_val_score(model, X_train, Y_train,cv=kfold,scoring="precision")
    cv_f1_results = model_selection.cross_val_score(model, X_train, Y_train,cv=kfold,scoring="f1")
    acc_score.append(cv_acc_results)
    auc_score.append(auc_score)
    pre_results.append(cv_pre_results)
    f1_results.append(cv_f1_results)
    
    result_col.loc[i]=[name,round(cv_acc_results.mean()*100,2),
                           round(cv_auc_results.mean()*100,2),
                           round(cv_pre_results.mean()*100,2),
                           round(cv_f1_results.mean()*100,2)]
    i+=1
    
result_col.sort_values(by=['ROC'],ascending=False)

,Algorithm,accuracy,ROC,Precision,f1_score
0,Logistic Regression,80.25,86.43,85.48,73.84
1,Kernel SVM,80.13,85.94,87.73,72.92
3,Multinomial NB,80.57,85.82,82.94,75.24
4,Random Forest,78.65,84.73,82.79,71.81
2,KNN,76.47,80.89,83.01,67.41
5,Decision Tree Classifier,74.22,73.74,71.41,69.58


# Hyperparameter Tuning

In [66]:
models_tuned = []

models_tuned.append(('Logistic Regression', LogisticRegression(solver='liblinear',C = 10, max_iter=500, random_state = 0)))
models_tuned.append(('SVC', SVC(C=1, kernel = 'linear', gamma=1, random_state = 0)))
models_tuned.append(('Kernel SVM', SVC(C=1, kernel = 'rbf', gamma=1, random_state = 0)))
models_tuned.append(('KNN', KNeighborsClassifier(n_neighbors = 5, metric = 'euclidean', p = 2,algorithm='brute')))
models_tuned.append(('Multinomial NB', MultinomialNB()))
models_tuned.append(('Decision Tree Classifier', DecisionTreeClassifier(criterion = 'gini', max_leaf_nodes=19, random_state = 0)))
models_tuned.append(('Random Forest', RandomForestClassifier(max_leaf_nodes = 24,max_features = 25,n_estimators = 50,criterion = 'entropy', random_state = 0)))

In [67]:
# Testing

acc_results_opt =[]
auc_results_opt =[]
pre_results_opt =[]
f1_results_opt =[]
names_opt = []

result_col_opt = ["Algorithm", "ROC AUC", "Accuracy", 'Precision', 'F1 Scores']
model_results_opt = pd.DataFrame(columns = result_col_opt)

i=0
# K- fold cross validation

for name, model in models_tuned:
    names_opt.append(name)
    kfold = model_selection.KFold(n_splits=10)
    
    cv_acc_results_opt = model_selection.cross_val_score(model, X_test, Y_test, 
                    cv = kfold, scoring="accuracy")
    cv_auc_results_opt = model_selection.cross_val_score(model, X_test, Y_test,
                    cv = kfold, scoring="roc_auc")
    cv_pre_results_opt = model_selection.cross_val_score(model, X_test, Y_test,
                    cv = kfold, scoring="precision")
    cv_f1_results_opt = model_selection.cross_val_score(model, X_test, Y_test,
                    cv = kfold, scoring="f1")
    acc_results_opt.append(cv_acc_results_opt)
    auc_results_opt.append(cv_auc_results_opt)
    pre_results_opt.append(cv_pre_results_opt)
    f1_results_opt.append(cv_f1_results_opt)
    model_results_opt.loc[i] = [name, 
                           round(cv_auc_results_opt.mean()*100,2),
                           round(cv_acc_results_opt.mean()*100,2),
                           round(cv_pre_results_opt.mean()*100,2),
                           round(cv_f1_results_opt.mean()*100,2)]
    i+=1
model_results_opt.sort_values(by = ['ROC AUC'], ascending=False)

,Algorithm,ROC AUC,Accuracy,Precision,F1 Scores
4,Multinomial NB,80.54,74.45,82.12,63.67
1,SVC,80.41,76.03,77.81,68.91
2,Kernel SVM,80.41,73.01,83.65,59.44
0,Logistic Regression,79.14,74.65,75.12,67.69
6,Random Forest,76.08,61.52,100.00,18.87
5,Decision Tree Classifier,61.23,65.07,72.55,42.01
3,KNN,59.38,60.01,100.00,12.84
